In [ ]:
import os
import open_clip
import torch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = 'ViT-B-32'
pretrained_datset = 'laion2b_s34b_b79k'

outputs_dir = os.path.join("outputs", f"{model_name}({pretrained_datset})")
os.makedirs(outputs_dir, exist_ok=True)

model, _, preprocess = open_clip.create_model_and_transforms(model_name, pretrained=pretrained_datset, device=device)
model.preprocess = preprocess

In [14]:
import clip
from utils import get_clip_features, get_dataset
from dimensionality_recuction import pca
import numpy as np
dataset_name = "STL10"

dataset, label_map = get_dataset(dataset_name, model.preprocess)
_, image_features, labels = get_clip_features(model, dataset, label_map, device,
                                                          os.path.join(outputs_dir, f"{dataset_name}_features"))

text_descriptions = [f"This is a photo of a {label}" for label in dataset.classes]
text_tokens = clip.tokenize(text_descriptions).cuda()
with torch.no_grad():
    text_features = model.encode_text(text_tokens).float()
    text_features /= text_features.norm(dim=-1, keepdim=True)



PCs, eigv, mean = pca(np.concatenate((text_features, image_features)))
text_embeddings = np.dot(text_features - mean, PCs)
image_embeddings = np.dot(image_features - mean, PCs)

/home/ariel/university/experiments/multimodal_experiments/alignment.py:91: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig(os.path.join(outputs_dir, "domain_alignment.png"))
/home/ariel/university/Internship/diffusion_venv/lib/python3.10/site-packages/IPython/core/pylabtools.py:170: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)


ValueError: Image size of 567x341229 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 640x480 with 1 Axes>

In [ ]:
idx = 0
x = image_embeddings[idx]
y_gt = labels[idx]
logits = x.unsqueeze(0) @ text_embeddings
y = logits.argmax(1)

